# Création d'une base de référence d'aggregats

In [1]:
# default_exp aggregates_read

In [2]:
# Activate multi-output in notebook
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [3]:
# export
import unittest

import yaml

from leximpact_socio_fisca_simu_etat.logger import logger

tc = unittest.TestCase()

In [4]:
from leximpact_socio_fisca_simu_etat.config import Configuration

config = Configuration()

## Lecture des aggregats

In [5]:
# export


class Aggregate:
    aggregats = None

    def __init__(self, aggregat_file):
        """
        ::aggregat_file:: YAML File containing the aggregats
        """
        with open(aggregat_file) as file:
            try:
                self.aggregats = yaml.safe_load(file)
            except yaml.YAMLError as exc:
                msg = f"Error reading yaml file: {exc}"
                logger.error(msg)
                raise exc

    def get_aggregate(self, year, variable_name, source, variable_type):
        try:
            return self.aggregats[year][variable_name][source][variable_type]
        except Exception:
            # logger.debug(f"Error reading {year}.{variable_name}.{variable_type}: {e}") #  as e
            return None

In [8]:
config.get("AGREGATS")

'/mnt/data-out/leximpact/agregats.yml'

In [9]:
aggregats = Aggregate(config.get("AGREGATS"))
aggregats.aggregats["2018"]["csg_deductible_salaire"]

{'ERFS': {'factor_to_2021': 1.173065725729275,
  'factor_to_2022': 1.173065725729275}}

In [10]:
tc.assertEqual(
    aggregats.get_aggregate("2020", "csg_salaire", "ccss", "sum"), 72628000000
)
tc.assertAlmostEqual(
    aggregats.get_aggregate("2018", "csg_deductible_salaire", "ERFS", "factor_to_2022"),
    1.1730,
    places=2,
)
tc.assertAlmostEqual(
    aggregats.get_aggregate("2018", "csg_imposable_salaire", "ERFS", "factor_to_2022"),
    1.1730,
    places=2,
)

In [11]:
tc.assertEqual(
    aggregats.get_aggregate("2018", "retraites", "POTE", "sum"), 299949516289
)
tc.assertEqual(aggregats.get_aggregate("2018", "retraites", "POTE", "nexistepas"), None)
tc.assertEqual(aggregats.get_aggregate("2018", "nexistepas", "POTE", "sum"), None)
tc.assertEqual(aggregats.get_aggregate("nexistepas", "retraites", "POTE", "sum"), None)